<a href="https://colab.research.google.com/github/VaibhavvvBarot/VaibhavvvBarot/blob/main/JsonCleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *Assign (Set Values)*

In [ ]:
import json
from google.colab import output
from IPython.display import display, HTML

In [22]:
# Define the valid keys for each type
LOGIC_CONTROL_KEYS = [
    "ValueType",
    "DocSourceName",
    "Value",
    "DataType",
    "ValueSubType",
    "DataSubType",
    "LookupType",
    "LookupName"
]

LOGIC_EXPRESSION_ADDITIONAL_KEYS = [
    "ConditionalOperator",
    "ID",
    "ExpressionType",
    "CompareToValue",
    "CompareToValueType",
    "CompareToDocSourceName",
    "CompareToDataType",
    "CompareToDataSubType",
    "LogicalOperator"
]

# Action-specific keys
ASSIGN_VALUE_OUTPUT_KEYS = ["AssignAction"]

def optimize_json(input_json):
    """Optimize JSON based on logicAction type"""
    optimized_json = {
        "logicAction": input_json.get("logicAction", "")
    }

    # Handle "Assign Value" logicAction
    if input_json.get("logicAction") == "Assign Value":
        # Process logicInstanceInputs
        if "logicInstanceInputs" in input_json:
            optimized_inputs = []
            for item in input_json["logicInstanceInputs"]:
                # Only keep keys that are present in the original item
                filtered_item = {}
                for key in LOGIC_CONTROL_KEYS:
                    if key in item:
                        filtered_item[key] = item[key]
                optimized_inputs.append(filtered_item)
            optimized_json["logicInstanceInputs"] = optimized_inputs

        # Process logicInstanceOutputs
        if "logicInstanceOutputs" in input_json:
            optimized_outputs = []
            for item in input_json["logicInstanceOutputs"]:
                # Keep standard LogicControl keys plus AssignAction
                filtered_item = {}
                # Add standard keys if present
                for key in LOGIC_CONTROL_KEYS:
                    if key in item:
                        filtered_item[key] = item[key]
                # Add action-specific keys if present
                for key in ASSIGN_VALUE_OUTPUT_KEYS:
                    if key in item:
                        filtered_item[key] = item[key]
                optimized_outputs.append(filtered_item)
            optimized_json["logicInstanceOutputs"] = optimized_outputs

    return optimized_json

# Get JSON input from user
json_text = input("Paste your JSON here: ")

try:
    input_json = json.loads(json_text)
    optimized_json = optimize_json(input_json)

    # Format the optimized JSON
    formatted_json = json.dumps(optimized_json, indent=2)

    # Clear previous output
    output.clear()

    # Create a text area with the optimized JSON for easy copying
    display(HTML(f"""
    <h3>Optimized JSON (Click inside the box, press Ctrl+A to select all, then Ctrl+C to copy):</h3>
    <textarea id="json-output" style="width:100%; height:250px; font-family:monospace; white-space:pre;">{formatted_json}</textarea>
    <script>
        // Auto-select the text for easy copying
        var textarea = document.getElementById('json-output');
        textarea.focus();
        textarea.select();
    </script>
    """))

    print("\nThe optimized JSON is ready to copy from the text area above.")

except Exception as e:
    print(f"Error: {str(e)}")


The optimized JSON is ready to copy from the text area above.
